# Football Players Analysis

Players analysis is going to be performed as follows:

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Get multiple **random samples** from the data set
  3. Get specific **one row** from the data set
  4. **Columns/fields categorization**
4. Columns groups **basic profiling** to better understand our data set:
  1. Player General Information columns basic profiling
  2. Argentina Team members
5. **Answer some business questions** to improve service
  1. Sum of IntCaps and IntGoals per Age
  2. Pick Top 10 players with good tackling 
  3. Categorize Strikers

Let's go for it:

## 1. PySpark environment setup

In [2]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## 2. Data source and Spark data abstraction (DataFrame) setup

In [3]:
playersDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("dataset.csv")

## 3. Data set metadata analysis
### A. Display schema and size of the DataFrame

In [4]:
from IPython.display import display, Markdown

playersDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % playersDF.count()))

root
 |-- UID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- NationID: integer (nullable = true)
 |-- Born: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- IntCaps: integer (nullable = true)
 |-- IntGoals: integer (nullable = true)
 |-- U21Caps: integer (nullable = true)
 |-- U21Goals: integer (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- AerialAbility: integer (nullable = true)
 |-- CommandOfArea: integer (nullable = true)
 |-- Communication: integer (nullable = true)
 |-- Eccentricity: integer (nullable = true)
 |-- Handling: integer (nullable = true)
 |-- Kicking: integer (nullable = true)
 |-- OneOnOnes: integer (nullable = true)
 |-- Reflexes: integer (nullable = true)
 |-- RushingOut: integer (nullable = true)
 |-- TendencyToPunch: integer (nullable = true)
 |-- Throwing: integer (nullable = true)
 |-- Corners: integer (nullable = true)
 |-- Crossing: integer (nullable = true)
 |-- Dribb

This DataFrame has **159541 rows**.

### B. Get multiple random samples from the data set

In [8]:
playersDF.cache() # optimization to make the processing faster
playersDF.sample(False, 0.1).take(5)

[Row(UID=1001575, Name='Liu Quankun', NationID=111, Born='17-02-1983', Age=33, IntCaps=5, IntGoals=0, U21Caps=0, U21Goals=0, Height=186, Weight=85, AerialAbility=2, CommandOfArea=2, Communication=2, Eccentricity=1, Handling=2, Kicking=3, OneOnOnes=3, Reflexes=2, RushingOut=3, TendencyToPunch=1, Throwing=1, Corners=2, Crossing=2, Dribbling=5, Finishing=3, FirstTouch=8, Freekicks=4, Heading=9, LongShots=6, Longthrows=1, Marking=11, Passing=4, PenaltyTaking=6, Tackling=10, Technique=4, Aggression=12, Anticipation=11, Bravery=10, Composure=5, Concentration=6, Vision=4, Decisions=7, Determination=12, Flair=4, Leadership=10, OffTheBall=3, Positioning=11, Teamwork=9, Workrate=9, Acceleration=6, Agility=12, Balance=8, Jumping=8, LeftFoot=8, NaturalFitness=9, Pace=6, RightFoot=20, Stamina=11, Strength=12, Consistency=12, Dirtiness=17, ImportantMatches=6, InjuryProness=5, Versatility=8, Adaptability=13, Ambition=13, Loyalty=13, Pressure=7, Professional=13, Sportsmanship=2, Temperament=2, Controv

### C. Get one row from the data set

In [6]:
playersDF.where("Name = 'Lionel Messi'")\
    .select("UID", "Name", "Born", "Age", "Height", "Weight", "IntCaps", "IntGoals")\
    .show()

+-------+------------+----------+---+------+------+-------+--------+
|    UID|        Name|      Born|Age|Height|Weight|IntCaps|IntGoals|
+-------+------------+----------+---+------+------+-------+--------+
|7458500|Lionel Messi|24-06-1987| 29|   170|    72|    113|      55|
+-------+------------+----------+---+------+------+-------+--------+




### D. Column categorization

The following could be a potential column categorization:

* **Player General Information columns:** *UID*, *Name*, *National ID*, *Born*, *Age*, *IntCaps*, *IntGoals*, *U21Caps*, *U21Goals*, *Height*, *Weight*, *CommandOfArea* ...
* **Player Ability  Information columns:** *Kicking*, *Reflexes*, *RushingOut*, *TendencyToPunch*, *Throwing* ...

## 4. Columns groups basic profiling to better understand our data set
### A. Player General Information columns basic profiling

In [6]:
from pyspark.sql.functions import when, count, col, countDistinct, desc, lit

generalColumns = ["UID", "Name", "Born", "Age", "Height", "Weight", "IntCaps", "IntGoals"]

print ("Summary of Player General Information:")
playersDF.select("UID", "Born", "Age", "Height", "Weight", "IntCaps", "IntGoals")\
    .summary("count", "min", "25%", "50%", "75%", "max")\
    .show(truncate=12)

print("Checking for nulls on general columns :")
playersDF.select([count(when(col(c).isNull(), c)).alias(c) for c in generalColumns]).show()

print("Checking amount of distinct values on general columns")
playersDF.select([countDistinct(c).alias(c) for c in generalColumns]).show()

print ("Summary of Player Attacking Ability Information:")
playersDF.select("AttackingMidCentral", "AttackingMidLeft", "AttackingMidRight")\
    .summary("count", "min", "25%", "50%", "75%", "max")\
    .show(truncate=12)

ageOccurrencesDF = playersDF.where('Age > 1')\
    .groupBy("Age")\
    .agg(count(lit(1)).alias("Count"))

print ("10 Most used Age:")

ageOccurrencesDF.orderBy(col("Count").desc()).show(n=10)

print ("10 Least used Age:")
ageOccurrencesDF.orderBy(col("Count").asc()).show(n=10)


Summary of Player General Information:
+-------+----------+----------+------+------+------+-------+--------+
|summary|       UID|      Born|   Age|Height|Weight|IntCaps|IntGoals|
+-------+----------+----------+------+------+------+-------+--------+
|  count|    159541|    159541|159541|159541|159541| 159541|  159541|
|    min|       510|01-01-1972|    14|   149|     0|      0|       0|
|    25%|  19022374|      null|    19|   175|     0|      0|       0|
|    50%|  41044730|      null|    23|   180|    68|      0|       0|
|    75%|  67232682|      null|    27|   185|    75|      0|       0|
|    max|1394673938|31-12-1999|    54|   210|   118|    166|      65|
+-------+----------+----------+------+------+------+-------+--------+

Checking for nulls on general columns :
+---+----+----+---+------+------+-------+--------+
|UID|Name|Born|Age|Height|Weight|IntCaps|IntGoals|
+---+----+----+---+------+------+-------+--------+
|  0|   0|   0|  0|     0|     0|      0|       0|
+---+----+----+-


### B. Argentina Team members

In [7]:
from pyspark.sql.functions import count, col, desc, lit, udf
from decimal import *
@udf
def Height(value):
    return str(value) + "cm"

@udf
def Weight(value):
    return str(value) + "kg"

ArgentinaTeamDF = playersDF.where("NationID = 1649")
print ("Summary of Argentina Team:")
ArgentinaTeamDF.select("UID", "Born", "Age", "Height", "Weight", "IntCaps", "IntGoals")\
    .summary("count", "min", "25%", "50%", "75%", "max")\
    .show(truncate=12)

print("Players in Argentina Team:")
ArgentinaTeamDF.select("Name", "Born", "Age",Height("Height"), Weight("Weight"), "IntCaps", "IntGoals")\
    .orderBy(col("IntGoals").desc())\
    .show(30)


Summary of Argentina Team:
+-------+----------+----------+----+------+------+-------+--------+
|summary|       UID|      Born| Age|Height|Weight|IntCaps|IntGoals|
+-------+----------+----------+----+------+------+-------+--------+
|  count|      8596|      8596|8596|  8596|  8596|   8596|    8596|
|    min|     12545|01-01-1984|  15|   153|     0|      0|       0|
|    25%|  14023214|      null|  21|   199|     0|      0|       0|
|    50%|  14077415|      null|  24|   199|    67|      0|       0|
|    75%|1394670531|      null|  48|   199|   102|      0|       0|
|    max|1394670531|31-12-1997|  48|   199|   102|    129|      55|
+-------+----------+----------+----+------+------+-------+--------+

Players in Argentina Team:
+--------------------+----------+---+--------------+--------------+-------+--------+
|                Name|      Born|Age|Height(Height)|Weight(Weight)|IntCaps|IntGoals|
+--------------------+----------+---+--------------+--------------+-------+--------+
|        L


## 5. Answer some business questions to improve service
### A. Sum of IntCaps and IntGoals per Age

In [24]:
from pyspark.sql.functions import count, max, col, desc, lit

ageGroupDF = playersDF.groupBy('Age') 
ageGroupDF\
    .agg(max("IntCaps").alias("IntCaps"), max("IntGoals").alias("IntGoals"))\
    .orderBy(col("IntCaps").desc(), col("IntGoals").desc(), col("Age").desc())\
    .show(ageGroupDF.count().count(), truncate=False)


+---+-------+--------+
|Age|IntCaps|IntGoals|
+---+-------+--------+
|35 |166    |65      |
|38 |161    |65      |
|31 |161    |61      |
|34 |153    |62      |
|33 |148    |57      |
|37 |146    |54      |
|43 |143    |4       |
|36 |138    |64      |
|30 |136    |53      |
|39 |135    |29      |
|40 |134    |12      |
|32 |133    |50      |
|29 |130    |55      |
|28 |121    |45      |
|41 |121    |16      |
|26 |95     |34      |
|27 |94     |35      |
|24 |70     |46      |
|25 |70     |31      |
|42 |67     |3       |
|45 |60     |10      |
|23 |58     |18      |
|22 |40     |11      |
|50 |39     |5       |
|21 |34     |22      |
|20 |31     |10      |
|18 |29     |11      |
|53 |25     |1       |
|44 |24     |1       |
|19 |20     |6       |
|48 |17     |1       |
|17 |13     |4       |
|16 |4      |1       |
|46 |1      |0       |
|54 |0      |0       |
|51 |0      |0       |
|49 |0      |0       |
|47 |0      |0       |
|15 |0      |0       |
|14 |0      |0       |
+---+------


### B. Pick Top 10 players with good tackling

In [17]:
from pyspark.sql.functions import count, sum, col, desc, lit

playersDF.select("Name", "Age", "Height", "Weight", "Tackling")\
    .orderBy(col("Tackling").desc())\
    .show(truncate=False)


+-------------------------+---+------+------+--------+
|Name                     |Age|Height|Weight|Tackling|
+-------------------------+---+------+------+--------+
|Javier Mascherano        |32 |174   |73    |20      |
|Laurent Koscielny        |30 |186   |75    |19      |
|Jan Mikula               |24 |183   |77    |19      |
|Radja Nainggolan         |28 |176   |65    |19      |
|Sokratis Papastathopoulos|28 |186   |85    |19      |
|Ljubomir Fejsa           |27 |183   |83    |18      |
|Dante                    |32 |187   |82    |18      |
|Sven Bender              |27 |186   |80    |18      |
|Giorgio Chiellini        |31 |187   |86    |18      |
|Kyriakos Papadopoulos    |24 |182   |79    |18      |
|Medhi Benatia            |29 |190   |92    |18      |
|Jérôme Boateng           |27 |192   |90    |18      |
|Tomás Rincón             |28 |175   |75    |18      |
|Benedikt Höwedes         |28 |187   |82    |18      |
|Domagoj Vida             |27 |184   |73    |18      |
|Pablo Zab


### C. Categorize Strikers

In [50]:
from pyspark.sql.functions import count, sum, col, desc, lit

strikersCategorizationDF = playersDF\
    .select("Name", "Striker")\
   .withColumn("StrikerSkill", when(col("Striker")>15,"1.Excellent")\
                               .when((col("Striker")>10) & (col("Striker")<=15),"2.Good")\
                               .when((col("Striker")>5) & (col("Striker")<=10),"3.Normal")\
                               .when((col("Striker")>0) & (col("Striker")<=5),"4.Bad")\
                               .otherwise("5.Worst"))
    
strikersCategorizationDF.cache() # optimization to make the processing faster
strikersCategorizationDF.groupBy("StrikerSkill")\
    .agg(count(lit(1)).alias("Count"))\
    .orderBy(col("StrikerSkill").asc())\
    .show()


+------------+------+
|StrikerSkill| Count|
+------------+------+
| 1.Excellent| 30656|
|      2.Good|  7712|
|    3.Normal|  4585|
|       4.Bad|116588|
+------------+------+

